🍳 Gen AI Kitchen Companion: Smart Recipes from What You Have
This project enhances home cooking by integrating GenAI to generate smart recipe suggestions based on ingredients users already have at home.

🔧 Capabilities Used:
Few-shot Prompting: Generate complete recipes from simple lists of ingredients.

Structured Output: Return recipe name, ingredients used, and step-by-step cooking instructions in JSON format.

Lightweight Interaction: No database needed—just smart prompt engineering and Google Gemini API for personalized assistance.


## 🧠 Problem Statement

Most recipe apps fail when you don’t have all the ingredients. Users need something smarter — a system that takes what’s in their pantry or fridge, understands their dietary preferences, and gives them easy-to-follow, personalized instructions.

The Gen AI Kitchen Companion solves this with intelligent prompting and structured generation techniques.


## 🔧 Gen AI Capabilities Used

| Capability | Description |
|------------|-------------|
| **Structured Output** | Recipe output is structured as JSON: ingredients, steps, time, etc. |
| **Few-shot Prompting** | Prompt tuned with examples for high-quality recipe results |
| **Function Calling (optional)** | Used to expand the recipe or fetch a shopping list if ingredients are missing |


In [1]:
import os
from google import genai
import numpy as np
from kaggle_secrets import UserSecretsClient
from sklearn.metrics.pairwise import cosine_similarity


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:623: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


## 🔐 API Key Management and Environment Setup

To integrate Google services such as Vertex AI or embedding models via API, it’s important to handle credentials securely, especially in public or collaborative environments like Kaggle. Here's how we manage API keys using Kaggle Secrets:

### ✅ Securely Fetching Google API Key from Kaggle Secrets

In [2]:
# ✅ Get API key from Kaggle secrets
GOOGLE_API_KEY = UserSecretsClient().get_secret("Google_API_Key")

# Set the Google API key in the environment variable
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

### 🧠 Generating Smart Recipes Using Few-Shot Prompting

This function uses **few-shot prompting** to guide the Gemini model in generating creative recipe suggestions based on the user's available ingredients.

It provides several example ingredient lists and corresponding recipe responses to help the model understand the desired format and style. The model then generates a relevant recipe for the user's current input using the Gemini 2.0 Flash model.


In [3]:
def get_smart_recipe(ingredients_list):
    # Convert list to readable string
    user_ingredients = ", ".join(ingredients_list)

    # Few-shot examples to guide the model
    prompt = f"""Suggest a recipe using available ingredients.

User: tomato, onion, garlic, turmeric
Assistant: Try making a simple tomato curry. Sauté chopped onion and garlic in oil, add chopped tomatoes and turmeric. Cook until soft and serve with rice.

User: paneer, bell pepper, onion, cumin
Assistant: You can make Paneer Tikka Masala. Sauté onions and cumin seeds, add bell peppers and paneer. Add masala and cook until tender.

User: rice, coconut, green chili, mustard seeds
Assistant: Make coconut rice! Cook rice, then fry mustard seeds and green chilies in oil. Mix in grated coconut and add to the rice.

User: {user_ingredients}
Assistant:"""

    # Load Gemini Pro model
    import google.generativeai as genai
    genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content(prompt)

    return response.text.strip()


### 🔍 Generating Recipe Embeddings for Similarity Matching

This function creates vector embeddings for ingredient lists or dish descriptions using the **Gemini 2.0 Flash model**.

These embeddings are useful for comparing recipes using **cosine similarity**, enabling the system to recommend recipes similar to the user's input. The function also includes error handling and debug logs to ensure robustness.


In [4]:
def get_recipe_embedding(text):
    import google.generativeai as genai
    genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
    
    try:
        # Get the embedding for the ingredient list or dish description
        embedding_prompt = f"Generate a vector representation for the ingredients or dish: {text}"
        model = genai.GenerativeModel("gemini-2.0-flash")
        embedding_response = model.generate_content(embedding_prompt)

        print("Embedding Response:", embedding_response.text.strip())  # Debugging line

        embedding_str = embedding_response.text.strip()
        if not embedding_str:
            print("⚠️ Warning: Embedding is empty.")
            return np.zeros(0)
        
        # Convert string to numpy array
        embedding = np.fromiter(map(float, embedding_str.split(',')), dtype=float)
        if embedding.size == 0:
            print("⚠️ Warning: Embedding array is empty.")
        return embedding

    except Exception as e:
        print(f"❌ Error generating embedding: {e}")
        return np.zeros(0)


### 🧠 Structured Recipe Generation with JSON Output

This function uses a prompt-based approach to generate a **structured recipe** from the user's input (such as ingredients or dish request). It leverages the **Gemini 2.0 Flash** model to return a JSON object with:

- `recipe_name`: Short name of the suggested dish  
- `ingredients`: List of required ingredients  
- `steps`: Step-by-step cooking instructions  
- `cuisine`: Suggested cuisine (e.g., Indian, Italian)  
- `meal_type`: Appropriate meal time (e.g., lunch, snack)  
- `prep_time`: Estimated preparation time in minutes  

The output is parsed from the model response into a valid JSON format for further use.


In [5]:
def get_structured_recipe(user_input):
    structured_prompt = f"""
You are a smart kitchen assistant. Based on the user's available ingredients or recipe request, return a recipe in structured JSON format with the following keys:

- recipe_name: short dish name
- ingredients: list of required ingredients
- steps: list of cooking instructions
- cuisine: suitable cuisine (e.g., Indian, Italian)
- meal_type: breakfast, lunch, dinner, snack, etc.
- prep_time: estimated preparation time in minutes

Respond **only** with valid JSON.

User input: "{user_input}"
JSON:
"""
    
    import google.generativeai as genai
    import os
    import json

    genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
    model = genai.GenerativeModel("gemini-2.0-flash")

    try:
        response = model.generate_content(structured_prompt)
        return json.loads(response.text)
    except json.JSONDecodeError:
        return {"error": "⚠️ Failed to parse structured recipe response."}

## 🔍 Recipe Retrieval Using Embeddings

To intelligently match user-provided ingredients or phrases to the most relevant recipe, we compute **cosine similarity** between the user's input and a database of recipe embeddings.

### 🧠 `find_similar_recipe()` Function

In [6]:
def find_similar_recipe(user_input, recipe_db):
    user_embedding = get_recipe_embedding(user_input)
    
    # Check if the embedding is valid
    if user_embedding.size == 0:
        print("Error: No valid embedding found for the user input.")
        return "Sorry, I couldn't find a matching recipe."

    recipe_embeddings = [recipe['embedding'] for recipe in recipe_db]
    
    # Calculate cosine similarity
    similarities = cosine_similarity([user_embedding], recipe_embeddings)
    most_similar_index = np.argmax(similarities)
    return recipe_db[most_similar_index]['recipe']


## 🧪 Main Script: Gen AI Kitchen Companion Demo

This is the interactive script for testing your Gen AI Kitchen Companion using multiple strategies:
- 🔍 **Embeddings + Vector Search**
- 🤖 **Few-shot Prompting**
- 📦 **Structured JSON Output**

### 🚀 Script

In [ ]:
if __name__ == "__main__":
    # Example recipe database with pre-calculated embeddings
    recipe_db = [
        {'recipe': 'Tomato Rice', 'embedding': get_recipe_embedding('tomato, rice, cumin, turmeric')},
        {'recipe': 'Paneer Bhurji', 'embedding': get_recipe_embedding('paneer, onion, tomato, spices')},
        {'recipe': 'Lemon Rice', 'embedding': get_recipe_embedding('lemon, rice, mustard seeds, curry leaves')},
        {'recipe': 'Mixed Vegetable Pulao', 'embedding': get_recipe_embedding('carrot, beans, peas, rice, garam masala')},
        {'recipe': 'Simple Khichdi', 'embedding': get_recipe_embedding('rice, moong dal, turmeric, salt')}
    ]

    while True:
        user_input = input("\n🧑‍🍳 What ingredients do you have? (or type 'exit'): ")
        if user_input.lower() == "exit":
            break

        # ✅ Few-shot Prompting
        basic_response = get_smart_recipe(user_input)
        print("\n👩‍🍳 Assistant (Few-shot Recipe Suggestion):", basic_response)

        # ✅ Embeddings + Vector Search
        similar_recipe = find_similar_recipe(user_input, recipe_db)
        print("🍲 Assistant (Similar Recipe Suggestion):", similar_recipe)

        # ✅ Structured Output / JSON Mode
        structured_json = get_structured_recipe(user_input)
        print("📦 Assistant (Structured Recipe Info in JSON):")
        print(structured_json)


Embedding Response: Here's a possible vector representation for the ingredients: tomato, rice, cumin, turmeric. This representation is based on a simple attribute-based approach.  You can think of it as a simplified feature vector.

**Conceptual Basis:**

We'll represent each ingredient based on these categories:

*   **Flavor Profile:** (Sweet, Savory, Spicy, Earthy, Bitter)
*   **Culinary Use:** (Base, Spice, Side, Garnish)
*   **Color:** (Red, White/Light, Yellow/Orange, Brown/Dark)
*   **Texture:** (Soft, Firm, Grainy, Powdery)
*   **Region of Origin:** (Mediterranean, Asian, South Asian, Latin American)

**Vector Representation:**

We'll use a 1/0 (or a scaled value) to indicate the presence or absence of a characteristic.

```
Ingredient Vector: [Flavor Profile, Culinary Use, Color, Texture, Region of Origin]
```

**Specific Vectors:**

*   **Tomato:** `[Sweet(0.7), Savory(0.5), Spicy(0.1), Earthy(0.2), Bitter(0.1), Base(0.8), Spice(0.2), Side(0.3), Garnish(0.5), Red(0.9), White/


🧑‍🍳 What ingredients do you have? (or type 'exit'):  apple,venigar



👩‍🍳 Assistant (Few-shot Recipe Suggestion): Okay, with the letters "a, p, p, l, e,  v, e, n, i, g, a, r", you can make **Apple Vinegar**.

Therefore, I suggest a recipe using **apple cider vinegar**:

**Apple Cider Vinegar Salad Dressing**

This is a very simple and versatile salad dressing you can make with just a few extra ingredients you likely have on hand.  It highlights the flavor of the apple cider vinegar.

**Ingredients:**

*   3 tablespoons apple cider vinegar
*   1 tablespoon olive oil (or other oil of your choice)
*   1 teaspoon honey (or maple syrup, or sugar - adjust to your preference)
*   Salt and pepper to taste

**Instructions:**

1.  **Combine:** In a small bowl, whisk together the apple cider vinegar, olive oil, and honey until well combined.
2.  **Season:** Add salt and pepper to taste.
3.  **Taste and Adjust:** Taste the dressing and adjust the ingredients as needed.  Add more honey if you want it sweeter, or more apple cider vinegar if you want it tangier.
4.  *